In [303]:
from langchain.agents import Tool, initialize_agent, AgentType, load_tools
from dotenv import load_dotenv
from langchain.utilities import (
    WikipediaAPIWrapper,
    GoogleSearchAPIWrapper,
)
from langchain.tools import WikipediaQueryRun, DuckDuckGoSearchRun
from langchain import OpenAI, Wikipedia

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
)
from langchain.chat_models import ChatOpenAI

In [304]:
load_dotenv()

True

In [305]:
myTopic = input("Enter your topic: ")

## Getting Keywords


In [306]:
wiki = WikipediaAPIWrapper()
google = GoogleSearchAPIWrapper()
duck = DuckDuckGoSearchRun()

In [307]:
llm_keywords = OpenAI(temperature=0.2)

In [308]:
# keyword_extractor_tools = load_tools(['wikipedia'],llm=llm_keywords)
keyword_extractor_tools = [
    Tool(
        name="Google Search",
        description="Useful when you want to get information from Google about single topic.",
        func=google.run,
    ),
    Tool(
        name="DuckDuckGo Search Evaluation",
        description="Useful to evaluate the results of Google Search and add any missing information about specific topic.",
        func=duck.run,
    ),
]

In [309]:
keyword_agent = initialize_agent(
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    agent_name="Keyword extractor",
    agent_description="You are a helpful AI that helps the user to get the important keyword list from the search results about specific topic.",
    llm=llm_keywords,
    tools=keyword_extractor_tools,
    verbose=True,
)

In [310]:
keyword_agent.agent.llm_chain.prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\nGoogle Search: Useful when you want to get information from Google about single topic.\nDuckDuckGo Search Evaluation: Useful to evaluate the results of Google Search and add any missing information about specific topic.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Google Search, DuckDuckGo Search Evaluation]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'

In [311]:
keyword_list = keyword_agent.run(
    f"Search about {myTopic} and use the results to get the important keywords related to {myTopic} to help to write a blog about {myTopic}."
)



> Entering new  chain...


 I need to find out what keywords are related to Messi
Action: Google Search
Action Input: "Messi"
Observation: Lionel Andrés Messi also known as Leo Messi, is an Argentine professional footballer who plays as a forward for Major League Soccer club Inter Miami and ... 479M Followers, 287 Following, 1063 Posts - See Instagram photos and videos from Leo Messi (@leomessi) LionelMessi · HT/WT. 5' 7", 159 lbs · Birthdate. 6/24/1987 (36) · Nationality. Argentina. Lionel Messi, 36, from Argentina ➤ Inter Miami CF, since 2023 ➤ Attack ➤ Market value: €35.00m ➤ * Jun 24, 1987 in Rosario, Argentina. Feb 1, 2018 ... Messi (Rosario, Argentina, 24 June 1987) is the greatest player of all time and FC Barcelona record setter. Trained at La Masia, ... Hit the follow button for all the latest on Lionel Messi! Fan Club for Snapchat: 400K | Instagram: 223K | DMs open for collabs - wearemessi@outlook. 2 Lionel Messi on the 2023 The World's Highest-Paid Athletes - Messi claimed the Ballon d'Or as the world

In [312]:
print(keyword_list)

Keywords related to Messi include: Lionel Andrés Messi, Leo Messi, Argentine professional footballer, Major League Soccer, Inter Miami, La Masia, Ballon d'Or, world's best soccer player, Major League Soccer, Inter Miami, South Florida, global icon, lifestyle brand, soccer legend, Leo Messi, Precision lifestyle clothing.


## Getting Title and SubTitle


In [313]:
title_llm = OpenAI()  # temperature=0.7

title_tools = [
    Tool(
        name="Intermediate Answer",
        description="Useful for when you need to ask with search about single topic.",
        func=google.run,
    ),
]

self_ask_with_search = initialize_agent(
    title_tools, title_llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)
title = self_ask_with_search.run(
    f"Suggest a titel for a blog about {myTopic}?",
)
subtitle = self_ask_with_search.run(
    f"Suggest a suitable subtitle for a blog about {myTopic}?",
)
print("title", title)
print("subtitle", subtitle)



> Entering new  chain...
 No.
So the final answer is: "The Magic of Messi: Celebrating the Phenomenal Footballer".

> Finished chain.


> Entering new  chain...
 No.
So the final answer is: "The Magic of Messi"

> Finished chain.
title "The Magic of Messi: Celebrating the Phenomenal Footballer".
subtitle "The Magic of Messi"


## Get search results


In [314]:
google_results = google.run(myTopic)
wiki_results = wiki.run(myTopic)
duck_results = duck.run(myTopic)
wiki_query_results = WikipediaQueryRun(api_wrapper=wiki).run(myTopic)
print("google_results", google_results)
print("wiki_results", wiki_results)

c:\Users\Peter\anaconda3\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\Peter\anaconda3\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


google_results Lionel Andrés Messi also known as Leo Messi, is an Argentine professional footballer who plays as a forward for Major League Soccer club Inter Miami and ... 479M Followers, 287 Following, 1063 Posts - See Instagram photos and videos from Leo Messi (@leomessi) LionelMessi · HT/WT. 5' 7", 159 lbs · Birthdate. 6/24/1987 (36) · Nationality. Argentina. Lionel Messi, 36, from Argentina ➤ Inter Miami CF, since 2023 ➤ Attack ➤ Market value: €35.00m ➤ * Jun 24, 1987 in Rosario, Argentina. Feb 1, 2018 ... Messi (Rosario, Argentina, 24 June 1987) is the greatest player of all time and FC Barcelona record setter. Trained at La Masia, ... Hit the follow button for all the latest on Lionel Messi! Fan Club for Snapchat: 400K | Instagram: 223K | DMs open for collabs - wearemessi@outlook. 2 Lionel Messi on the 2023 The World's Highest-Paid Athletes - Messi claimed the Ballon d'Or as the world's best soccer player for a record seventh time. 3 days ago ... Lionel Messi has finalized his de

## Summarise the search results


In [315]:
# summarize the results separately
prompt = """Please Provide a summary of the following essay
The essay is: {essay}.
The summary is:"""
prompt_template = PromptTemplate(
    template=prompt,
    input_variables=["essay"],
)
summarize_llm = OpenAI(temperature=0)

In [316]:
summary_chain = LLMChain(
    llm=summarize_llm,
    prompt=prompt_template,
)
google_summary = summary_chain.run(essay=google_results)
wiki_summary = summary_chain.run(essay=wiki_results)
duck_summary = summary_chain.run(essay=duck_results)
wiki_query_summary = summary_chain.run(essay=wiki_query_results)

In [317]:
text_spitter = RecursiveCharacterTextSplitter(
    separators=[".", "\n", "\t", "\r", "\f", "\v", "\0", "\x0b", "\x0c"],
    chunk_size=1000,
    chunk_overlap=500,
)

In [318]:
docs = text_spitter.create_documents(
    [google_results, wiki_results, duck_results, wiki_query_results]
)
num_docs = len(docs)
summary_chain2 = load_summarize_chain(
    llm=summarize_llm,
    chain_type="map_reduce",
)

In [319]:
tot_summary = summary_chain2.run(docs)

In [320]:
summary_tools = [
    Tool(
        name="Wikipedia",
        func=wiki.run,
        description="Search engine useful when you want to get information from Wikipedia about single topic.",
    ),
    Tool(
        name="Intermediate Answer",
        func=WikipediaQueryRun(api_wrapper=wiki).run,
        description="Search engine useful for when you need to ask with search",
    ),
    Tool(
        name="Google Search",
        description="Search engine useful when you want to get information from Google about single topic.",
        func=google.run,
    ),
    Tool(
        name="DuckDuckGo Search",
        description="Search engine useful when you want to get information from DuckDuckGo about single topic.",
        func=duck.run,
    ),
]

In [321]:
summary_agent = initialize_agent(
    summary_tools,
    llm=summarize_llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    agent_name="Summary Agent",
    verbose=True,
)

In [322]:
tot_summary2 = summary_agent.run(
    f"can you provide me a summary about {myTopic} from each search engine separately? \ then use this information to combine all the summaries together to get a blog about {myTopic}."
)



> Entering new  chain...
 I need to get summaries from each search engine and then combine them together.
Action: Wikipedia
Action Input: Messi
Observation: Page: Lionel Messi
Summary: Lionel Andrés Messi (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] (listen); born 24 June 1987), also known as Leo Messi, is an Argentine professional footballer who plays as a forward for Major League Soccer club Inter Miami and captains the Argentina national team. Widely regarded as one of the greatest players of all time, Messi has won a record seven Ballon d'Or awards and a record six European Golden Shoes, and in 2020 he was named to the Ballon d'Or Dream Team. Until leaving the club in 2021, he had spent his entire professional career with Barcelona, where he won a club-record 34 trophies, including ten La Liga titles, seven Copa del Rey titles and the UEFA Champions League four times. With his country, he won the 2021 Copa América and the 2022 FIFA World Cup. A prolific goalscorer and creative

## Create a chain to write the first draft of the blog


In [323]:
prompt_writer = """You are an expert online blogger with expert writing skills and I want you to only write out the breakdown of each section of the blog on the topic of {topic} 
using the following information:
keywords: {keywords}.
The title is: {title}.
The subtitle is: {subtitle}.
google results: {google_results}.
wiki results: {wiki_results}.
duck results: {duck_results}.
wiki query results: {wiki_query_results}.
google summary: {google_summary}.
wiki summary: {wiki_summary}.
duck summary: {duck_summary}.
wiki query summary: {wiki_query_summary}.
The results summary is: {summary}.
The final blog will be {wordCount} words long, with an introduction at the beginning and a conclusion at the end of the blog.
use the following template to write the blog:
[BLOG]
[title]
[subtitle]
[introduction]
[body in bullet points]
[summary]
[END BLOG]
"""

prompt_writer_template = PromptTemplate(
    template=prompt_writer,
    input_variables=[
        "topic",
        "title",
        "subtitle",
        "google_results",
        "wiki_results",
        "duck_results",
        "wiki_query_results",
        "google_summary",
        "wiki_summary",
        "duck_summary",
        "wiki_query_summary",
        "summary",
        "keywords",
        "wordCount",
    ],
)
writer_llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k")

In [324]:
writer_chain = LLMChain(
    llm=writer_llm,
    prompt=prompt_writer_template,
    verbose=True,
)

In [325]:
draft1 = writer_chain.run(
    topic=myTopic,
    title=title,
    subtitle=subtitle,
    google_results=google_results,
    wiki_results=wiki_results,
    duck_results=duck_results,
    wiki_query_results=wiki_query_results,
    google_summary=google_summary,
    wiki_summary=wiki_summary,
    duck_summary=duck_summary,
    wiki_query_summary=wiki_query_summary,
    summary=tot_summary + tot_summary2,
    keywords=keyword_list,
    wordCount=3000,
)



> Entering new  chain...
Prompt after formatting:
You are an expert online blogger with expert writing skills and I want you to only write out the breakdown of each section of the blog on the topic of Messi 
using the following information:
keywords: Keywords related to Messi include: Lionel Andrés Messi, Leo Messi, Argentine professional footballer, Major League Soccer, Inter Miami, La Masia, Ballon d'Or, world's best soccer player, Major League Soccer, Inter Miami, South Florida, global icon, lifestyle brand, soccer legend, Leo Messi, Precision lifestyle clothing..
The title is: "The Magic of Messi: Celebrating the Phenomenal Footballer"..
The subtitle is: "The Magic of Messi".
google results: Lionel Andrés Messi also known as Leo Messi, is an Argentine professional footballer who plays as a forward for Major League Soccer club Inter Miami and ... 479M Followers, 287 Following, 1063 Posts - See Instagram photos and videos from Leo Messi (@leomessi) LionelMessi · HT/WT. 5' 7", 159 l

In [326]:
print(draft1)

[BLOG]
The Magic of Messi: Celebrating the Phenomenal Footballer
The Magic of Messi

Introduction:
Lionel Andrés Messi, also known as Leo Messi, is an Argentine professional footballer who has captured the hearts of millions of fans around the world. With his incredible skills and unmatched talent, Messi has established himself as one of the greatest players in the history of the sport. In this blog, we will delve into the magic of Messi and celebrate his phenomenal career.

Body:

1. Early Life and Career:
- Messi was born on June 24, 1987, in Rosario, Argentina.
- He joined Barcelona's youth academy, La Masia, at the age of 13.
- Messi made his competitive debut for Barcelona at the age of 17 in October 2004.
- He quickly established himself as an integral player for the club and helped Barcelona achieve the first treble in Spanish football in the 2008-09 season.

2. Record-Breaking Achievements:
- Messi has won a record seven Ballon d'Or awards, cementing his status as the world's b

In [327]:
print("len draft1", len(draft1))

len draft1 2265


In [328]:
# Put the text in draft1 in text file
with open("draft1.txt", "w") as f:
    f.write(draft1)

## Evaluate the first draft
